# Fine-tuning a Large Language Model

In this lecture we will be looking at how to fine-tune an existing pre-trained language model.

## Learning outcomes
* You will learn how to download a pre-trained model and a training dataset from Hugging Face.
* You will learn how to fine-tune the downloaded model with the dataset using Hugging Face trl library and the supervised fine-tuning (SFT) method.
* You will learn how to use the fine-tuned model to generate text based on user input / prompts.
* You will learn how to upload the fine-tuned model to your own Hugging Face repository so that it can be used later or shared with other users.

## Prerequistes
* You will need the following free accounts: Google, Hugging Face and Weights & Biases. You may use your existing accounts or create new accounts for the purposes of this course.
* We will use the [Hugging Face](https://huggingface.co/) libraries: transformers (for models), datasets (for datasets), trl (for training). We will also store the fine-tuned models in a Hugging Face repository.
* Training is done using [Google Colab](https://colab.research.google.com/), which provides free access to Jupyter notebooks backed with a GPU compute required for fine-tuning.
* For monitoring the training run we will use [Weights & Biases](https://wandb.ai/)


## Fine-tuning

Let's first install some pre-requisites using Python's package manager pip

In [1]:
!pip install transformers peft accelerate
!pip install -U datasets
!pip install -q trl xformers wandb einops sentencepiece bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 100.1 MB/s eta 0:00:00
 

Then we need to import the required libraries

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import torch, wandb
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

We will download a pre-trained large language model from Hugging Face and a dataset to train the model with. Below we assign these to variables we will use later. We will also set the name of the repository and model for the fine-tuned model.

In [38]:
# Pre trained model
model_name = "facebook/opt-350m"

# Dataset name
dataset_name = "databricks/databricks-dolly-15k"

# Hugging face repository link to save fine-tuned model(Create new repository in huggingface,copy and paste here)
new_model = "litian0/opt350m"

To access your Hugging Face account, you need to log in. First go to your Hugging Face account, click *Settings* and select *Access Tokens*. Create a new token and copy the token. Then execute the below login command and when asked paste an access token.  

In [39]:
notebook_login()

Let's then download a subset of the dataset we want to use. Below we limit the dataset to the first 10,000 examples in order to save time. In real life you would probably use the full dataset.

In [40]:
dataset = load_dataset(dataset_name, split="train[0:100]")
dataset["context"][0]

"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."

Let's then download the model. We first create a config object for quantization of the model using bitsandbytes. Bitsandbytes enables accessible large language models via k-bit quantization for PyTorch.

We also need to download the tokenizer.

In [41]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

Below we set the access token to Waights & Biases. You should copy your access token from your account at [https://wandb.ai](https://wandb.ai).

In [42]:
#monitering login
wandb.login(key="") # Add your WANDB key here
run = wandb.init(project='Fine tuning opt 350m', job_type="training", anonymous="allow")

Then we'll create a configuration for the lo-rank adaptation method we will use.

In [43]:
peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"]
)

We need to set the training arguments for the training run.

In [44]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    save_steps=1000,
    logging_steps=30,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.3,
    group_by_length=True,
    lr_scheduler_type="linear",
    report_to="wandb",
)

Finally we create the trainer object that uses supervised fine-tuning (SFT) as the training method.

In [45]:
# Before creating the trainer, add this preprocessing step
def combine_text(example):
    return {
        "text": f"Instruction: {example['instruction']}\n"
               f"Context: {example['context']}\n"
               f"Response: {example['response']}"
    }

dataset = dataset.map(combine_text)

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
)


<ipython-input-45-f0eb0922bf55>:12: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Then, we can execute the training run. This will approximately 8 hours using the T4 GPU available in Colab and the dataset of 10,000 samples we downloaded.

In [46]:
# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=6, training_loss=6.712455113728841, metrics={'train_runtime': 32.1656, 'train_samples_per_second': 3.109, 'train_steps_per_second': 0.187, 'total_flos': 73445511659520.0, 'train_loss': 6.712455113728841, 'epoch': 0.9230769230769231})

In [47]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁
train/global_step,▁
total_flos,73445511659520.0
train/epoch,0.92308
train/global_step,6
train_loss,6.71246
train_runtime,32.1656
train_samples_per_second,3.109
train_steps_per_second,0.187


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=1024, bias=False)
              )
              (lora_embedding_A): Par

In [52]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    # More specific prompt template
    prompt = (
        "You are a knowledgeable physics teacher. Please provide a clear and accurate answer "
        f"to the following question:\n\n{user_prompt.strip()}\n\n"
        "Answer:"
    )

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_config = {
        "max_new_tokens": 100,
        "temperature": 0.3,  # Lower temperature for more focused responses
        "top_p": 0.9,
        "do_sample": True,
        "repetition_penalty": 1.1,
        "pad_token_id": tokenizer.eos_token_id,
    }

    _ = model.generate(
        **inputs,
        streamer=streamer,
        **generation_config
    )


In [64]:
stream("what is newtons 3rd law and its formula")



Newtons 3rd law is the law of the universe which states that the universe has two dimensions, one of which is called the physical plane and the other of which is called the cosmological plane. It is the law of the universe in which the universe is divided into two parts, one being the physical plane and the other being the cosmological plane. The physical plane is the physical plane of the universe while the cosmological plane is the physical plane of the universe.



I changed the base model from mistralai/Mistral-7B-v0.3 to facebook/opt-350m and the dataset from vicgalle/alpaca-gpt4 to databricks/databricks-dolly-15k. This was done to use a smaller model with fewer parameters, which requires less computational resources, so the train time is less.

However, the model is generating a nonsensical response about "physical" and "cosmological" planes, which is completely unrelated to Newton's Third Law. This indicates that the model, facebook/opt-350m, is not well-suited for this task, even with the prompt engineering we've done. It's likely that this model was not trained on sufficient physics data to understand the concept.

In [ ]:
# This will fail due to cuda out of memeory issue. Need to add quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [66]:
model.push_to_hub("litian0/opt")
tokenizer.push_to_hub("litian0/opt")

model.safetensors:   0%|          | 0.00/293M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/litian0/opt/commit/2a73da22a227af6b47e60c905d87fa719ff774d7', commit_message='Upload tokenizer', commit_description='', oid='2a73da22a227af6b47e60c905d87fa719ff774d7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/litian0/opt', endpoint='https://huggingface.co', repo_type='model', repo_id='litian0/opt'), pr_revision=None, pr_num=None)